In [ ]:
import requests, json, csv
import time, datetime, os
import pandas as pd

dt = datetime.datetime.now()
yearList = range(2016, dt.year+1)
monthList = range(1, 13)
isStock = True


#下載檔案.
def getWebmsg(yy, mm, sid):
    date = '%d%02d01' %(yy, mm)
    res = requests.get('http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date={}&stockNo={}'.format(date, sid))
    return res


#寫入檔案.
def writeCsv(sid, directory, filename, rows):
    outputFile = open(directory+filename, 'w', newline='', encoding='big5')
    outputWriter = csv.writer(outputFile)
    for row in rows:
        outputWriter.writerow(row)
    print(sid, filename, "done")
    outputFile.close()


#新建資料夾.
def makedirs(sid):
    directory = 'D:/StockData/HistoryData/'+sid+'/'
    if not os.path.isdir(directory):
        os.makedirs(directory)


#從1101到9999的股票代號開始抓取每月交易資料.
for sid in range(1101, 10000):
    for yy in yearList:
        for mm in monthList:
            if (dt.year == year and month > dt.month):
                break
            #抓取交易資料,若有內容則寫入檔案,若抓取失敗則停頓3秒再重新抓取,失敗超過20次則跳過此筆資料.
            errorNum = 0
            while True:
                try:
                    res = getWebmsg(yy, mm, sid)
                    rows = csv.reader(res.text)
                    for row in rows:
                        if row == []:
                            isStock = False
                        break
                    if not isStock:
                        break
                    
                    directory = 'D:/StockData/HistoryData/{}/'.format(sid)
                    filename = '{}{:02d}.csv'.format(yy, mm)
                    makedirs(sid)
                    writeCsv(sid, directory, filename, rows)
                    errorNum = 0
                    break                        
                except:
                    errorNum += 1
                    print('{}{}{:02d} error {} times'.format(sid, yy, mm, errorNum))
                    time.sleep(3)
                    if errorNum > 20:
                        isStock = False
                        break
                    continue
            if not isStock:
                break
            time.sleep(3)
        if not isStock:
            isStock = True
            print('{} no data'.format(sid))
            time.sleep(3)
            break
            

#將取得的資料依各股及月份做清洗整理
sidList = os.listdir(r'D:/StockData/HistoryData')

for sid in sidList:
    dataList = os.listdir('D:/StockData/HistoryData/{}'.format(sid))
    
    for dataName in dataList:
        try:
            writedata = open('D:/StockData/ForAnalysis/{}/{}'.format(sid, dataName), 'w', newline='', encoding='utf-8-sig')
            writer = csv.writer(writeData)
            openData = open('D:/StockData/HistoryData/{}/{}'.format(sid, dataName), 'r', encoding='big5') 
            rows = csv.reader(openData)
            tempData = []
            
            for row in rows:
                if row == [] or row == ['', '']:
                    continue
                tempData.append(row)
            if not tempData[0] == ['日期']:
                del tempData[0]
            openData.close()
            
            for n in range(len(tempData)//9):
                writer.writerow(tempData[0: 9])
                del tempData[0: 9]
            writeData.close()
            
        except:
            print('{} {} error'.format(sid, dataName))
            openData.close()
            writeData.close()
            os.remove('D:/StockData/ForAnalysis/{}.csv'.format(sid))
    
    print('{} done'.format(sid))